## Principal Components Analysis(PCA)

- PCA requiresno assumptions about the distribution(s) of the dataset.
- PCA has a high compression rate(with good fidelity when information is uncompressed).
- There is no need for labels in the dataset as PCA works using only the inputs.
- PCA has a strong methematical background.
- There isno need to normalize the data before applying PCA.

In [1]:
Pkg.add("MultivariateStats")
using MultivariateStats

INFO: Cloning cache of MultivariateStats from https://github.com/JuliaStats/MultivariateStats.jl.git
INFO: Installing MultivariateStats v0.3.1
INFO: Package database updated
INFO: METADATA is out-of-date — you may not have the latest version of MultivariateStats
INFO: Use `Pkg.update()` to get the latest versions of your packages
INFO: Precompiling module MultivariateStats.


In [55]:
X = readcsv("C:\\dev\\Julia\\JuliaStudy\\Data files-selected\\OnlineNewsPopularity\\OnlineNewsPopularity.csv")[2:end, 2:end];

In [56]:
N, n = size(X)
n -= 1

I = Array(Float64, N, n)
O = X[:, end]

for j in 1:n
    for i in 1:N
        I[i, j] = Float64(X[i, j])
    end
end

In [57]:
ind = (O .>= 10000) # cases of "viral" result
O_bin = falses(N)
O_bin[ind] = true

true

In [58]:
z = round(Int64, N*0.8)
ind = randperm(N)

39644-element Array{Int64,1}:
 24405
  3998
  7016
 39405
 30620
 31871
 20668
 27515
 30156
 19842
 10017
 16333
 20769
     ⋮
  4300
 38172
 11414
 31344
 33670
 27434
 22646
  5070
  8765
 10315
 29240
 18961

In [59]:
Itr = I[ind[1:z],:]
Ite = I[ind[(z+1):end],:]
Otr = O[ind[1:z]] # this will be useful later on
Ote = O[ind[(z+1):end]] # this will be useful later on

7929-element Array{Any,1}:
  1700
  2200
  2100
 48000
   854
  2100
   759
  4400
  4600
   942
  4700
  4100
  3500
     ⋮
  1500
   850
  1300
  1100
  1700
  1800
   404
   970
   642
  2000
  2700
  2200

In [60]:
M = fit(PCA, Itr'; maxoutdim = 10) # limit the total number of variables in reduced feature set to 10

PCA(indim = 59, outdim = 4, principalratio = 0.99383)

In [61]:
Jtr = transform(M, Itr')'

31715×4 Array{Float64,2}:
      1.30131e5   58610.6        -28991.1   -22807.6  
     -1.74876e5  -52409.4         15087.6     -787.17 
     -1.26972e5      -1.19289e5   28936.6     4548.7  
      1.48767e5   95801.1        -45104.0   -24661.9  
      1.12035e5   21269.3        -21280.8   -17067.0  
  86918.0        -29502.1         -4998.66   -9643.41 
  99106.5         -4821.08       -17339.2    -2800.07 
      1.45809e5   89701.3        -40200.9   -31866.8  
  82763.0        -37809.0         -3139.17   -5499.5  
      1.25366e5   50968.8        -16693.7   -19525.4  
      1.41955e5   82124.3        -36255.9   -31578.7  
  97466.0         -8163.89       -15792.0    -3700.11 
      1.71553e5       1.47702e5  -38933.4   -12400.2  
      ⋮                                               
  78262.2        -47104.7         -4416.62    3149.09 
  70198.9        -63320.0          7740.05   -9205.26 
      1.16681e5   30750.1        -26301.8   -12860.2  
 -88820.5        -39730.4         14532

In [62]:
Jte = transform(M, Ite')'

7929×4 Array{Float64,2}:
   96282.0             -6.16823     34984.4      3831.01 
       1.50768e5        1.17478e5   33795.9     -5789.9  
 -717664.0          81410.8         -6172.66    -6375.84 
       1.12619e5    29397.1           435.095   11981.6  
       1.10251e5    19655.4         -8611.54   -19836.8  
   46342.5             -1.11691e5   21998.7     -1099.52 
   71513.1         -60690.5         -3518.16    13671.4  
   21767.5        -161771.0         33319.4     11946.5  
       2.07804e5        2.20967e5  -58542.2    -31836.3  
       1.74855e5   148656.0        -59468.5    -36773.3  
       1.20959e5    40133.6        -27038.3    -10468.8  
   46659.2             -1.11293e5   18177.4      4182.57 
       1.70611e5        1.49127e5  -14232.9    -28480.2  
       ⋮                                                 
      -1.93874e5   -91474.2         23112.6      6982.9  
   86055.1         -31478.9         -3548.22   -13993.1  
   33678.5             -1.36239e5   30894.4    

## Feature Evaluation and Selection

### Overview of the methodology

- Fisher's Discriminat Ratio(FDR)
- Index of Dscernibility(latest oepn0source version: DID)
- Similarity Metrics
    - Correlation
    - Cosine similarity
    - Jaccard and other binary similarity metrics.

In [63]:
function cossim(x::Array, y::Array)
    # Cosine similarity function
    nx = norm(x)
    ny = norm(y)
    
    if (nx == 0) | (ny == 0)
        return NaN
    else
        return dot(x, y) / (nx * ny)
    end
end

cossim (generic function with 1 method)

 at In[45]:3 overwritten at In[63]:3.


In [64]:
# an Array of Cosine Simliarity
V = Array(Float64, n);

In [65]:
for i in 1:n
    V[i] = cossim(Itr[:, i], Otr)
end

In [66]:
abs(V[1:5]), mean(abs(V))

([0.233057,0.264048,0.206322,0.0379392,0.0457274],0.16353460718574878)

In [67]:
th = 0.2
feat_ind = (abs(V) .>= th)
sum(feat_ind)

25

In [68]:
show(find(feat_ind))

[1,2,3,7,11,12,23,24,25,26,27,42,44,45,46,47,48,49,50,51,52,53,54,55,58]

In [71]:
# Using julia for feature evaluation and selection using DID
ind = (O .> 10000)
O_bin = falses(N)
O_bin[ind] = true

true

In [72]:
Otr = Array(O_bin[ind[1:z]])
Ote = Array(O_bin[ind[(z + 1):end]])

LoadError: LoadError: BoundsError: attempt to access 39644-element BitArray{1} at index [Bool[false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,false,false,false,fals

In [70]:
Otr = Array(O_bin[ind[1:z]])
Ote = Array(O_bin[ind[(z+1):end]])

7929-element Array{Bool,1}:
 false
 false
 false
  true
 false
 false
 false
 false
 false
 false
 false
 false
 false
     ⋮
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false